## Session 2: Manipulating data structures

Today, we begin with a few more complicated operations from [Lists](https://ocaml.org/manual/5.2/api/List.html) that we did not cover last time.

> `filter : ('a -> bool) -> 'a list -> 'a list`
>
> &emsp; `filter f l` returns all the elements of the list `l` that satisfy the predicate `f`. The order of the elements in the input list is preserved.

In [1]:
let rec filter f = function
    | [] -> [] 
    | x :: xs -> 
        if (f x) then
            x :: (filter f xs)
        else
            filter f xs

val length : 'a list -> int = <fun>


Like `map` and `mapi`, we have `filter` and `filteri` for the slightly more complicated situations.

> `filteri : (int -> 'a -> bool) -> 'a list -> 'a list`
>
> &emsp; Same as `List.filter`, but the predicate is applied to the index of the element as first argument (counting from 0), and the element itself as second argument.

In [ ]:
let rec filteri' f i = function
    | [] -> [] 
    | x :: xs -> 
        if (f x i) then
            x :: (filter f xs (i + 1))
        else
            filter f xs (i + 1)

In [ ]:
let filteri xs = filteri' f 0 xs

For example, if we want to remove all elements in the list that at positions `3, 6, 9, ...` (in OCaml, the modulo operator is `%`):

In [ ]:
let filter_3m xs = filteri (fun i x -> i % 3 = 0) xs

In [ ]:
filter_3m [0; 1; 2; 3; 4; 5; 6; 7; 8; 9; 10; 11; 12]

Now, let us solve a few exercise questions, directly and using the library functions we learned today and yesterday.

1. (a) Write a function `product` so that `product [a; b; c; d]`
   computes the product `a * b * c * d`.

   (b) Write a tail-recursive version of `product`.
   
   (c) What does `product [-1;-2;0;1;2;3]` return?  Is your
   implementation as efficient as possible?


Part (a) is simply an instance of the fold function, but let's complete it in the straightforward way, in order to do part (b).

In [ ]:
let product xs = List.fold (*) 1 xs

In [ ]:
let product = function
  | [] -> 1
  | x :: xs -> x * (product xs)

Part (b) is done via an accumulator.

In [ ]:
let product' acc = function
  | [] -> acc
  | x :: xs -> product' (x * acc) xs

In [ ]:
let product xs = product' 1 xs

For part (c), we know that everything times zero is zero, so the function should immediately return zero when it sees a zero in the list, to have better time efficiency. Note that we could use OCaml's pattern matching to do the comparison for us.

In [ ]:
let product' acc = function
  | [] -> acc
  | 0 :: _ -> 0
  | x :: xs -> product' (x * acc) xs

In [ ]:
let product xs = product' 1 xs

2. Write a function `count` so that `count x xs` returns the number of
   times `x` occurs in `xs`.

We start with the simplest way of implementing it.

In [ ]:
let rec count x = function
  | [] -> 0
  | y :: ys -> 
      if y = x then
        1 + (count x xs)
      else
        count x xs

It is not hard to see that this function could be easily turned into a tail-recursive one, using the accumulator to store the count.

In [ ]:
let rec count' x acc = function
  | [] -> acc
  | y :: ys -> 
      if y = x then count' x (acc + 1) ys
      else count' x acc ys

In [ ]:
let count x xs = count' x 0 xs


3. Write a function `repeat` so that `repeat n x` returns a list with
   `n` copies of `x`.  For example, `repeat 4 10` should return the
   list `[10; 10; 10; 10]`.

We get `repeat` by calling `List.init` with a constant function that always returns `x`.

In [ ]:
let repeat n x = List.init n (fun _ -> x)


4. Write a function `shorter` so that `shorter xs ys` returns the
   shorter of the two lists `xs` and `ys`.  Can you avoid examining
   every cons cell in each list?

To do this, we first define a function that compares the lengths of `xs` and `ys`. Note that this is provided in the standard library:
> `compare_lengths : 'a list -> 'b list -> int`
> 
> Compare the lengths of two lists. `compare_lengths l1 l2` is equivalent to `compare (length l1) (length l2)`, except that the computation stops after reaching the end of the shortest list.

In [ ]:
let rec compare_lengths xs ys = 
  match (xs, ys) with
    | [] , [] -> 0
    | [] , _ :: _ -> -1
    | _ :: _ , [] -> 1
    | _ :: xs, _ :: ys -> compare_lengths xs ys

In [ ]:
let compare xs ys = 
  if compare_lengths >= 0 then xs
  else ys

5. Write a function `is_sorted` to detect if a list is already in sorted order.

This is another example of fancy pattern matching: we could match for a singleton list, or a list with at least two elements.

In [ ]:
let is_sorted = function
  | [] -> true
  | [_] -> true
  | x1 :: x2 :: xs -> (x1 <= x2) && (sorted (x2 :: xs))

Recall that we can represent `sum` with `List.fold_left`:

In [ ]:
let sum xs = List.fold (+) 0 xs

We can represent a wide range of functions with `fold_left`, for example, the `length` function, as long as we know how to define a suitable `f`:

In [ ]:
let length xs = List.fold (fun acc _ -> acc + 1) 0 xs

All data types have their corresponding versions of `map` and `fold`. Mapping a function `f` over a tree is simply distributing this function over all children nodes of the tree.

In [ ]:
type 'a tree = Lf | Br of 'a * 'a tree * 'a tree

In [ ]:
let rec map_tree f = function 
  | Lf -> Lf
  | Br (a, t1, t2) -> Br (f a, map_tree f t1, map_tree f t2)

To better understand `fold`, we look at the special cases, `count` and `sum`.

In [ ]:
let rec count_tree = function 
  | Lf -> 0
  | Br (a, t1, t2) -> 1 + (count_tree t1) + (count_tree t2)

In [ ]:
let rec sum_tree = function 
  | Lf -> 0
  | Br (a, t1, t2) -> a + (sum_tree t1) + (sum_tree t2)

Now, we can generalize the pattern here: at the base case, we return some initial value; at recursive case, we use a function `f : 'a -> 'b -> 'b -> 'b` to combine the values of the current node's element and the outcomes from the recursive calls on the two subtrees.

In [ ]:
let rec fold_tree init f = function  
  | Lf -> init
  | Br (a, t1, t2) -> f a (fold_tree init f t1) (fold_tree init f t2)

Using `fold_tree` to write the `depth` function:

In [ ]:
let depth = function  
  | Lf -> 0
  | Br (a, t1, t2) -> 1 + max (depth t1) (depth t2)

In [ ]:
let depth tr = fold_tree 0 (fun a r1 r2 -> 1 + max r1 r2)